## Hello World with SLURM

### PyISlurm: Python Interface to Slurm https://pyslurm.github.io

In [ ]:
# Load the "magic" commands from ipyslurm 
%load_ext ipyslurm

In [ ]:
%slogin login-2

In [ ]:
%%sbatch
#!/usr/bin/env bash
#SBATCH --job-name=Rmpi
#SBATCH --nodes=1
#SBATCH --time=00:05:00     
#SBATCH --ntasks-per-node=48
#SBATCH --cpus-per-task=1
#SBATCH --output=rmpi-%j.log
#SBATCH --partition=nu-wide

cd $SLURM_SUBMIT_DIR

module load mpich/3.2.1-gcc-7.2.0
module load gcc/7.2.0
module load R/3.5.1
module list

RFILE="test.R"
WORKERS=$(($SLURM_NTASKS-1))

# Create R script
cat > $RFILE << EOF
# Load library
library(Rmpi)

# In case R exits unexpectedly, have it automatically clean up
# resources taken up by Rmpi (slaves, memory, etc...)
.Last <- function(){
   if (is.loaded("mpi_initialize")){
     if (mpi.comm.size(1) > 0){
       print("Please use mpi.close.Rslaves() to close workers.")
       mpi.close.Rslaves()
     }
     print("Please use mpi.quit() to quit R")
     .Call("mpi_finalize")
   }
}

# Spawn workers
mpi.spawn.Rslaves(nslaves=$WORKERS)

# Execute a command on each worker
mpi.remote.exec(paste("I am",mpi.comm.rank(),"of",mpi.comm.size()), ret=FALSE)
mpi.close.Rslaves()
mpi.quit()
EOF

echo "Using $SLURM_JOB_NUM_NODES nodes"
mpirun -n 1 Rscript $RFILE # Only works with 1 worker (up to 64 CPUs)
#mpiexec -n 24 Rscript $RFILE

In [ ]:
%%sbash
squeue -u $USER